In [1]:
import gc
import os
import sys
import json
import psutil
import logging
import warnings
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf

sys.path.append("..")

from src.data import DataExtractor, DataProcessor

warnings.filterwarnings("ignore")
pd.set_option("display.float_format", "{:,.4f}".format)

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

logger.info("Environment ready — pandas: %s", pd.__version__)

2025-10-15 19:01:04,707 - INFO - Environment ready — pandas: 2.3.3


In [2]:
symbol = "BTCUSDT"  # e.g., BTCUSDT, ETHUSDT
data_type = "metrics"  # source archive type label used in filenames

data_folder = Path.cwd().parent / "data"
downloaded_folder = data_folder / "downloaded"
raw_folder = data_folder / "raw"
symbol_folder = raw_folder / symbol / data_type
symbol_folder.mkdir(parents=True, exist_ok=True)

# print(f"Symbol: {symbol}, data folder: {symbol_folder}")


### Data extraction and chunking

This section extracts CSV files from downloaded ZIP archives, converts rows to typed columns and slices the resulting stream into fixed-size Parquet parts. Each part is sized to balance on-disk compactness and in-memory loading costs so downstream analyses can either load parts selectively or concatenate them into a single DataFrame when resources allow.

The code is written as reusable functions so you can re-run extraction, inspect parts, or re-chunk quickly during experiments.

In [3]:
ROWS = 250_000  # for klines
# ROWS = 1_000_000  # target rows per parquet part

def get_cols_and_dtypes_by_df(df: pd.DataFrame):
	"""Get column names and dtypes from a DataFrame."""
	cols = df.columns.tolist()
	dtypes = df.dtypes.apply(lambda x: x.name).tolist()
	return cols, dtypes

parts = DataExtractor.extract_and_chunk(symbol, data_type, ROWS, time_col="create_time")

2025-10-15 19:01:13,031 - INFO - Saved 2 parts to /home/bloodrain/Desktop/Deep_Learning/Crypto_Market_Analyzer/data/raw/BTCUSDT/metrics


### Resource usage inspection

Quickly measure on-disk size, the DataFrame's logical memory footprint and the RSS delta observed when reading each Parquet part. This helps decide whether you can safely load all data into RAM or whether you must adopt chunked/out-of-core processing. The helper below returns a concise summary and prints per-part metrics.

In [4]:
def sizeof_fmt(num, suffix="B"):
	for unit in ["", "Ki", "Mi", "Gi", "Ti"]:
		if abs(num) < 1024.0:
			return f"{num:3.1f}{unit}{suffix}"
		num /= 1024.0
	return f"{num:.1f}Pi{suffix}"


def rss_bytes():
	return psutil.Process(os.getpid()).memory_info().rss


def analyze_parts(parts):
	"""Return summary dict and print per-part metrics."""
	if not parts:
		print(f"No parquet parts found in {symbol_folder}")
		return {}

	rows = []
	disk_sizes = []
	logical_mem = []
	rss_deltas = []
	overheads = []

	for p in parts:
		size_on_disk = p.stat().st_size
		before = rss_bytes()
		df = pd.read_parquet(p, engine="pyarrow")
		after = rss_bytes()
		row_count = len(df)
		mem_bytes = df.memory_usage(deep=True).sum()
		delta_rss = max(after - before, 0)
		overhead = delta_rss / mem_bytes if mem_bytes else 0

		rows.append(row_count)
		disk_sizes.append(size_on_disk)
		logical_mem.append(mem_bytes)
		rss_deltas.append(delta_rss)
		overheads.append(overhead)

		print(f"{p.name}: rows={row_count:,}, disk={sizeof_fmt(size_on_disk)}, ")
		print(f"  logical_mem={sizeof_fmt(mem_bytes)}, rss_delta={sizeof_fmt(delta_rss)}, overhead={overhead:,.2f}x")

		del df
		gc.collect()

	total_rows = sum(rows)
	total_disk = sum(disk_sizes)
	total_logical = sum(logical_mem)
	total_rss = sum(rss_deltas)
	avg_overhead = sum(overheads) / len(overheads) if overheads else 0
	avg_mem_per_row = total_rss / total_rows if total_rows else 0
	est_total_single_df = int(total_rss * 1.10)

	summary = {
		"parts": len(parts),
		"total_rows": total_rows,
		"total_disk": total_disk,
		"total_logical": total_logical,
		"total_rss": total_rss,
		"avg_overhead": avg_overhead,
		"avg_mem_per_row": avg_mem_per_row,
		"estimated_memory_one_df": est_total_single_df,
	}

	print("\nSUMMARY")
	print(f"parts: {summary['parts']}")
	print(f"total rows: {summary['total_rows']:,}")
	print(f"total disk size: {sizeof_fmt(summary['total_disk'])}")
	print(f"total logical mem: {sizeof_fmt(summary['total_logical'])}")
	print(f"total rss delta: {sizeof_fmt(summary['total_rss'])}")
	print(f"avg overhead factor: {summary['avg_overhead']:.2f}x")
	print(f"avg mem per row: {sizeof_fmt(summary['avg_mem_per_row'])}/row")
	print(f"estimated memory to load as one DataFrame (≈+10% overhead): {sizeof_fmt(summary['estimated_memory_one_df'])}")

	return summary


parts = sorted(symbol_folder.glob("part*.parquet"))
summary = analyze_parts(parts)


part0001.parquet: rows=250,000, disk=11.7MiB, 
  logical_mem=26.7MiB, rss_delta=39.5MiB, overhead=1.48x
part0002.parquet: rows=155,940, disk=8.8MiB, 
  logical_mem=16.7MiB, rss_delta=29.8MiB, overhead=1.79x

SUMMARY
parts: 2
total rows: 405,940
total disk size: 20.6MiB
total logical mem: 43.4MiB
total rss delta: 69.2MiB
avg overhead factor: 1.63x
avg mem per row: 178.8B/row
estimated memory to load as one DataFrame (≈+10% overhead): 76.1MiB


### Continuity checks for time-series integrity

Confirm the 1-minute cadence of the `open_time` column across and between Parquet parts. The checks detect duplicates, non-monotonic steps and missing rows; they produce a compact issues table suitable for filtering and further remediation (e.g., interpolation or dropping corrupted segments). The function below returns a DataFrame of issues.

In [9]:
df = pd.concat([pd.read_parquet(p, engine="pyarrow") for p in parts], ignore_index=True)
df = DataProcessor.sort_by_time(df, time_col="create_time")
res = DataProcessor.check_time_continuity(df, time_col="create_time", delta_ms=60_000 * 5)

In [11]:
print("Missing intervals (if any):", res["missing"])
print("Duplicate intervals (if any):", res["duplicate"])
print("Non-monotonic intervals (if any):", res["too_close"])

Missing intervals (if any): [{'prev_time_ms': 1638593700000, 'cur_time_ms': 1638594300000, 'diff_ms': 600000, 'missing_rows': 1}, {'prev_time_ms': 1638595200000, 'cur_time_ms': 1638596100000, 'diff_ms': 900000, 'missing_rows': 2}, {'prev_time_ms': 1638748500000, 'cur_time_ms': 1638835200000, 'diff_ms': 86700000, 'missing_rows': 288}, {'prev_time_ms': 1639594800000, 'cur_time_ms': 1639595400000, 'diff_ms': 600000, 'missing_rows': 1}, {'prev_time_ms': 1640358000000, 'cur_time_ms': 1640358600000, 'diff_ms': 600000, 'missing_rows': 1}, {'prev_time_ms': 1640820000000, 'cur_time_ms': 1640820600000, 'diff_ms': 600000, 'missing_rows': 1}, {'prev_time_ms': 1640966400000, 'cur_time_ms': 1640967000000, 'diff_ms': 600000, 'missing_rows': 1}, {'prev_time_ms': 1694507700000, 'cur_time_ms': 1694508900000, 'diff_ms': 1200000, 'missing_rows': 3}, {'prev_time_ms': 1708090200000, 'cur_time_ms': 1708128000000, 'diff_ms': 37800000, 'missing_rows': 125}, {'prev_time_ms': 1709510100000, 'cur_time_ms': 170951

In [10]:
df = df.iloc[105120:]
if df.empty:
	print("DataFrame is empty — ensure parts exist or run extraction.")
else:
	display(df.head())

	df_without_time = df.drop(["open_time", "close_time"], axis=1, errors="ignore")
	print(f"\n{'='*10} STATS {'='*10}\n")
	display(df_without_time.describe().T)

	print(f"\n{'='*10} INFO {'='*10}\n")
	df.info()

	print(f"\n{'='*10} MISSING STATS {'='*10}\n")
	missing_per_column = pd.DataFrame({
		"count": df.isnull().sum(),
		"percent": (df.isnull().sum() / df.shape[0] * 100).round(2)
	})
	display(missing_per_column)

	print(f"\n{'='*10} DUPLICATES STATS {'='*10}\n")
	duplicates_per_column = pd.DataFrame({
		"count": df.shape[0] - df.nunique(),
		"percent": ((df.shape[0] - df.nunique()) / df.shape[0] * 100).round(2)
	})
	display(duplicates_per_column)

	print(f"\n{'='*10} TOP 5 MOST FREQUENT VALUES {'='*10}\n")
	for col in df.columns:
		if df[col].nunique() == df.shape[0]:
			continue
		counts = df[col].value_counts(normalize=False).head(5)
		percents = (df[col].value_counts(normalize=True).head(5) * 100).round(2)
		combined = pd.DataFrame({
			"Value": counts.index,
			"Count": counts.values,
			"Percent": percents.values
		})
		print(f"Column: {col}")
		display(combined)

	sample = df if len(df) <= 100_000 else df.iloc[-100_000:]
	sample["Date"] = pd.to_datetime(sample["open_time"], unit="ms")
	sample = sample[["Date", "open", "high", "low", "close", "volume"]].set_index("Date")
	mpf.plot(
		sample,
		type="candle",
		style="yahoo",
		title=f"{symbol} Candlestick Chart (sample)",
		ylabel="Price",
		ylabel_lower="Volume",
		volume=True,
		figratio=(12, 4),
		tight_layout=True
	)

	summary = {
		"symbol": symbol,
		"parts": len(parts),
		"rows": int(df.shape[0]),
		"first_time": str(pd.to_datetime(int(df.open_time.min()), unit="ms")) if not df.empty else None,
		"last_time": str(pd.to_datetime(int(df.open_time.max()), unit="ms")) if not df.empty else None,
	}
	with open(symbol_folder / "summary.json", "w") as fh:
		json.dump(summary, fh, indent=2)
	print("Wrote summary.json to symbol folder")


,create_time,symbol,sum_open_interest,sum_open_interest_value,count_toptrader_long_short_ratio,sum_toptrader_long_short_ratio,count_long_short_ratio,sum_taker_long_short_vol_ratio
105120,1669941300000,BTCUSDT,129899,"2,203,052,072.7892",NaN,NaN,1.2095,1.2905
105121,1669941600000,BTCUSDT,129935,"2,205,201,662.0270",NaN,NaN,1.2133,1.3344
105122,1669941900000,BTCUSDT,130010,"2,207,689,447.3168",NaN,NaN,1.2193,2.4968
105123,1669942200000,BTCUSDT,129935,"2,206,104,482.5448",NaN,NaN,1.2223,0.7096
105124,1669942500000,BTCUSDT,129722,"2,204,652,468.1330",NaN,NaN,1.2340,2.1647



========== STATS ==========



,count,mean,std,min,25%,50%,75%,max
create_time,"300,820.0000","1,715,086,859,857.4094","26,068,401,905.0244","1,669,941,300,000.0000","1,692,502,725,000.0000","1,715,102,850,000.0000","1,737,664,875,000.0000","1,760,227,200,000.0000"
sum_open_interest,"300,820.0000","88,432.7260","12,666.5981",0.0000,"79,787.0000","86,518.0000","95,374.5000","139,463.0000"
sum_open_interest_value,"300,820.0000","5,256,046,583.5516","2,663,746,696.5155",0.0000,"2,991,226,045.3201","4,792,975,583.6818","7,592,638,861.8657","12,767,006,555.5914"
count_toptrader_long_short_ratio,"297,102.0000",1.4076,0.5103,0.5338,0.9909,1.3280,1.7476,3.6749
sum_toptrader_long_short_ratio,"297,136.0000",1.4332,0.3004,0.9029,1.1900,1.3573,1.6192,2.5486
count_long_short_ratio,"300,782.0000",1.3806,0.5640,0.4139,0.9278,1.2759,1.7754,3.7696
sum_taker_long_short_vol_ratio,"300,820.0000",1.1236,0.6587,0.0239,0.6949,0.9813,1.3869,63.6341



========== INFO ==========

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300820 entries, 105120 to 405939
Data columns (total 8 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   create_time                       300820 non-null  int64  
 1   symbol                            300820 non-null  object 
 2   sum_open_interest                 300820 non-null  int64  
 3   sum_open_interest_value           300820 non-null  float64
 4   count_toptrader_long_short_ratio  297102 non-null  float64
 5   sum_toptrader_long_short_ratio    297136 non-null  float64
 6   count_long_short_ratio            300782 non-null  float64
 7   sum_taker_long_short_vol_ratio    300820 non-null  float64
dtypes: float64(5), int64(2), object(1)
memory usage: 18.4+ MB

========== MISSING STATS ==========



,count,percent
create_time,0,0.0000
symbol,0,0.0000
sum_open_interest,0,0.0000
sum_open_interest_value,0,0.0000
count_toptrader_long_short_ratio,3718,1.2400
sum_toptrader_long_short_ratio,3684,1.2200
count_long_short_ratio,38,0.0100
sum_taker_long_short_vol_ratio,0,0.0000


...

Column: count_long_short_ratio


,Value,Count,Percent
0,2.2500,3,0.0000
1,1.5680,2,0.0000
2,1.7461,2,0.0000
3,0.8029,2,0.0000
4,1.0332,2,0.0000


Column: sum_taker_long_short_vol_ratio


,Value,Count,Percent
0,1.5167,4,0.0000
1,0.7140,4,0.0000
2,0.5869,4,0.0000
3,0.6798,4,0.0000
4,0.6909,4,0.0000


KeyError: 'open_time'